In [1]:
import json, bm25s, sys, torch, os, time
sys.path.append("../")
from transformers import AutoModelForMaskedLM, AutoTokenizer
from splade.models.transformer_rep import Splade
from tqdm import tqdm

/home/lamdo/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_type_or_dir = "/scratch/lamdo/splade_maxsim_ckpts/splade_maxsim_100k/debug/checkpoint/model"

model = Splade(model_type_or_dir, agg="max")
model.eval()
tokenizer = AutoTokenizer.from_pretrained(model_type_or_dir)
reverse_voc = {v: k for k, v in tokenizer.vocab.items()}

In [3]:
with open("/scratch/lamdo/erukg_cache_custom_trained_combined_references_nounphrase_v9-1.json") as f:
    phrases = json.load(f)

In [4]:
phrase_corpus = phrases["phrase_vocab"]
tokenized_corpus = [[reverse_voc[idx] for idx in tokenizer.encode(phrase, add_special_tokens=False)] for phrase in tqdm(phrase_corpus)]

100%|██████████| 221144/221144 [00:07<00:00, 27685.17it/s]


In [9]:
phrases["phrase_vocab"][:10]

['review',
 'children',
 'students',
 'design',
 'impact',
 'covid 19',
 'data',
 'analysis',
 'education',
 'china']

In [5]:
retriever = bm25s.BM25()
retriever.index(tokenized_corpus)

In [7]:
query_phrase = "text search quest is voice engine"
# query_phrase_tokenized = [reverse_voc[idx] for idx in tokenizer.encode(query_phrase, add_special_tokens=False)]
query_phrase_tokenized = ['pre', '##train', '##ed', '##ped', 'language', 'linguistic', 'type', 'vs', 'pl']

start = time.time()
results, scores = retriever.retrieve([query_phrase_tokenized], k=20)
end = time.time()

print(end - start)

for i in range(results.shape[1]):
    doc, score = results[0, i], scores[0, i]
    print(f"Rank {i+1} (score: {score:.2f}): {phrase_corpus[doc]}")

0.011720657348632812
Rank 1 (score: 8.01): pretrained language model
Rank 2 (score: 8.01): pretrained language models
Rank 3 (score: 7.25): pretrained cnn
Rank 4 (score: 7.25): pretrained weights
Rank 5 (score: 7.25): pretrained sequence
Rank 6 (score: 7.25): pretrained model
Rank 7 (score: 7.25): pretrained models
Rank 8 (score: 7.25): pretrained transformers
Rank 9 (score: 7.15): large pretrained language models
Rank 10 (score: 6.90): pretrain
Rank 11 (score: 6.38): pretrained bert model
Rank 12 (score: 5.82): pretraining
Rank 13 (score: 5.18): vs
Rank 14 (score: 5.03): bert pretraining
Rank 15 (score: 4.69): unsupervised pretraining
Rank 16 (score: 4.69): pretrained word embeddings
Rank 17 (score: 4.57): language models pre
Rank 18 (score: 4.57): language model pre
Rank 19 (score: 4.50): linguistic
Rank 20 (score: 4.42): domain adaptive pretraining


In [33]:
phrase_corpus[175933]

'machine learning machine'